- number of direct flood/flash flood fatalities per county since 1996
- estimated monetary crop damage

In [50]:
import os
import sys
import pandas as pd
import io
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
) 

In [51]:
pd.set_option('display.max_columns', None)


In [52]:
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/climate_risk/flood/loss/noaa/downloaded_files/all_events/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'all_noaa_storm_events_ca.csv'


In [53]:
noaa_storm_event_data = pd.read_csv('all_noaa_storm_events_ca.csv')
noaa_storm_event_data

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,FLOOD_CAUSE,CATEGORY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,199707,27,2032,199707,27,2032,1048394,5617146,CALIFORNIA,6,1997,July,Thunderstorm Wind,C,65,RIVERSIDE,PSR,27-JUL-97 20:32:00,MST,27-JUL-97 20:32:00,0,0,0,0,NaN,NaN,NaN,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLYTHE,NaN,NaN,BLYTHE,33.6200,-114.6000,33.6200,-114.6000,NaN,Thunderstorm wind gust was measured at the Bly...,PDC
1,199707,28,1700,199707,28,1700,2070479,5622118,CALIFORNIA,6,1997,July,Flash Flood,C,93,SISKIYOU,MFR,28-JUL-97 17:00:00,PST,28-JUL-97 17:00:00,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAPPY CAMP,NaN,NaN,HAPPY CAMP,NaN,NaN,NaN,NaN,NaN,FOREST SERVICE REPORTED A FLASH FLOOD ON GRANI...,PDC
2,199707,27,1600,199707,27,1800,2070480,5622119,CALIFORNIA,6,1997,July,Hail,C,93,SISKIYOU,MFR,27-JUL-97 16:00:00,PST,27-JUL-97 18:00:00,0,0,0,0,NaN,NaN,NaN,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,S,ETNA,8.0,S,ETNA,41.3300,-122.9000,41.3300,-122.9000,NaN,NaN,PDC
3,199707,27,1600,199707,27,1800,2070480,5622120,CALIFORNIA,6,1997,July,Flash Flood,C,93,SISKIYOU,MFR,27-JUL-97 16:00:00,PST,27-JUL-97 18:00:00,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,S,ETNA,4.0,S,ETNA,NaN,NaN,NaN,NaN,NaN,1.5 INCHES OF RAIN IN 2 HRS RESULTED IN MINOR ...,PDC
4,199707,30,1600,199707,30,1600,2070481,5622121,CALIFORNIA,6,1997,July,Flash Flood,C,93,SISKIYOU,MFR,30-JUL-97 16:00:00,PST,30-JUL-97 16:00:00,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,S,FT JONES,2.0,S,FT JONES,NaN,NaN,NaN,NaN,NaN,2 INCHES OF RAIN IN 30 MIN. RESULTED IN 2 FT. ...,PDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26959,202004,26,900,202004,30,1900,162838,983437,CALIFORNIA,6,2020,April,Excessive Heat,Z,566,IMPERIAL COUNTY WEST,PSR,26-APR-20 09:00:00,PST-8,30-APR-20 19:00:00,0,0,0,0,0.00K,0.00K,County Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strong high pressure over the Desert Southwest...,NaN,CSV
26960,202010,26,330,202010,26,345,152864,920678,CALIFORNIA,6,2020,October,High Wind,Z,568,CHIRIACO SUMMIT,PSR,26-OCT-20 03:30:00,PST-8,26-OCT-20 03:45:00,0,0,0,0,20.00K,0.00K,Department of Highways,35.00,ES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"During the morning hours on October 26th, a st...","Following a cold frontal passage, strong gusty...",CSV
26961,202008,15,1230,202008,15,1234,152203,916656,CALIFORNIA,6,2020,August,Tornado,C,35,LASSEN,REV,15-AUG-20 12:30:00,PST-8,15-AUG-20 12:34:00,0,0,0,0,0.00K,0.00K,NWS Storm Survey,NaN,NaN,NaN,NaN,EF1,0.43,137.0,NaN,NaN,NaN,NaN,3.0,WSW,PLUMAS,3.0,WSW,PLUMAS,39.7313,-120.1284,39.7301,-120.1209,Dry conditions and anomalously low fuel moistu...,A mesoanticyclone embedded within a deep pyroc...,CSV
26962,202008,15,1304,202008,15,1308,152203,916709,CALIFORNIA,6,2020,August,Tornado,C,35,LASSEN,REV,15-AUG-20 13:04:00,PST-8,15-AUG-20 13:08:00,0,0,0,0,0.00K,0.00K,NWS Storm Survey,NaN,NaN,NaN,NaN,EF1,0.09,20.0,NaN,NaN,NaN,NaN,32.0,ENE,COYOTEVILLE,32.0,ENE,COYOTEVILLE,39.7239,-120.1303,39.7226,-120.1299,Dry conditions and anomalously low fuel moistu...,A mesoanticyclone embedded within a deep pyroc...,CSV


In [54]:
noaa_storm_event_data.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [67]:
columns = [
'BEGIN_YEARMONTH', 'BEGIN_DAY', 'EVENT_ID',
'YEAR', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'DEATHS_DIRECT',
'DEATHS_INDIRECT', 'DAMAGE_CROPS','BEGIN_LAT', 'BEGIN_LON'
]

noaa_storm_columns = noaa_storm_event_data[columns]
noaa_storm_columns

,BEGIN_YEARMONTH,BEGIN_DAY,EVENT_ID,YEAR,EVENT_TYPE,CZ_TYPE,CZ_FIPS,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_CROPS,BEGIN_LAT,BEGIN_LON
0,199707,27,5617146,1997,Thunderstorm Wind,C,65,0,0,NaN,33.6200,-114.6000
1,199707,28,5622118,1997,Flash Flood,C,93,0,0,NaN,NaN,NaN
2,199707,27,5622119,1997,Hail,C,93,0,0,NaN,41.3300,-122.9000
3,199707,27,5622120,1997,Flash Flood,C,93,0,0,NaN,NaN,NaN
4,199707,30,5622121,1997,Flash Flood,C,93,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26959,202004,26,983437,2020,Excessive Heat,Z,566,0,0,0.00K,NaN,NaN
26960,202010,26,920678,2020,High Wind,Z,568,0,0,0.00K,NaN,NaN
26961,202008,15,916656,2020,Tornado,C,35,0,0,0.00K,39.7313,-120.1284
26962,202008,15,916709,2020,Tornado,C,35,0,0,0.00K,39.7239,-120.1303


In [56]:
noaa_storm_columns.EVENT_TYPE.unique()

array(['Thunderstorm Wind', 'Flash Flood', 'Hail', 'Wildfire',
       'High Surf', 'Heavy Rain', 'Dust Storm', 'Heat', 'Tornado',
       'Lightning', 'Heavy Snow', 'Flood', 'Winter Storm', 'Rip Current',
       'Funnel Cloud', 'High Wind', 'Waterspout', 'Cold/Wind Chill',
       'Winter Weather', 'Storm Surge/Tide', 'Strong Wind', 'Dust Devil',
       'Dense Fog', 'Tropical Storm', 'Frost/Freeze', 'Ice Storm',
       'Coastal Flood', 'Debris Flow', 'Avalanche', 'Blizzard', 'Drought',
       'Extreme Cold/Wind Chill', 'Tsunami', 'Excessive Heat',
       'Dense Smoke', 'Sneakerwave', 'Freezing Fog',
       'Astronomical Low Tide'], dtype=object)

## They is a small percentage of the data with lat & lon entries, so we will use county entries within the CZ_TYPE and CZ_FIPS

In [69]:
flood_types = [    
    'Flash Flood',
    'High Surf', 
    'Heavy Rain',  
    'Flood',
    'Funnel Cloud', 
    'Waterspout',
    'Coastal Flood',  
    'Tsunami'
]

cz_type_county = ['C']

# Filter for flood events based on event type
flood_events = noaa_storm_columns[noaa_storm_columns['EVENT_TYPE'].isin(flood_types)]

# Further filter for county events (CZ_TYPE == 'C')
flood_county_events = flood_events[flood_events['CZ_TYPE'].isin(cz_type_county)]

# Extract the month from the END_YEARMONTH column
# Assuming END_YEARMONTH is in the format YYYYMM, the month would be the last two characters
flood_county_events = flood_county_events.copy()  # To avoid SettingWithCopyWarning
flood_county_events['MONTH'] = flood_county_events['BEGIN_YEARMONTH'].astype(str).str[-2:]
flood_county_event_filtered = flood_county_events.drop(columns=['BEGIN_YEARMONTH', 'BEGIN_LAT',	'BEGIN_LON'])
# Display the updated DataFrame
flood_county_event_filtered


,BEGIN_DAY,EVENT_ID,YEAR,EVENT_TYPE,CZ_TYPE,CZ_FIPS,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_CROPS,MONTH
1,28,5622118,1997,Flash Flood,C,93,0,0,NaN,07
3,27,5622120,1997,Flash Flood,C,93,0,0,NaN,07
4,30,5622121,1997,Flash Flood,C,93,0,0,NaN,07
7,10,5622040,1997,Flash Flood,C,93,0,0,NaN,06
9,23,5600706,1997,Heavy Rain,C,83,0,0,NaN,07
...,...,...,...,...,...,...,...,...,...,...
26446,18,932481,2020,Heavy Rain,C,35,0,0,0.00K,11
26500,15,915118,2020,Flash Flood,C,37,0,0,0.00K,08
26622,18,930949,2020,Flash Flood,C,7,0,0,1.00K,11
26623,18,930952,2020,Flood,C,89,0,0,0.00K,11


In [73]:
flood_county_deaths = flood_county_event_filtered
flood_county_deaths['TOTAL_FATALITIES'] = flood_county_event_filtered['DEATHS_DIRECT'] + flood_county_event_filtered['DEATHS_INDIRECT']
flood_county_deaths.TOTAL_FATALITIES.unique()

array([0, 3, 1, 2], dtype=int64)

In [75]:
selected_columns = ['EVENT_ID']
flood_county_event_drop_duplicates = flood_county_deaths.duplicated(subset=selected_columns, keep='first')
flood_county_event_drop_duplicates
duplicate_count = flood_county_event_drop_duplicates.sum()
duplicate_count

0

In [74]:
flood_county_deaths

,BEGIN_DAY,EVENT_ID,YEAR,EVENT_TYPE,CZ_TYPE,CZ_FIPS,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_CROPS,MONTH,TOTAL_FATALITIES
1,28,5622118,1997,Flash Flood,C,93,0,0,NaN,07,0
3,27,5622120,1997,Flash Flood,C,93,0,0,NaN,07,0
4,30,5622121,1997,Flash Flood,C,93,0,0,NaN,07,0
7,10,5622040,1997,Flash Flood,C,93,0,0,NaN,06,0
9,23,5600706,1997,Heavy Rain,C,83,0,0,NaN,07,0
...,...,...,...,...,...,...,...,...,...,...,...
26446,18,932481,2020,Heavy Rain,C,35,0,0,0.00K,11,0
26500,15,915118,2020,Flash Flood,C,37,0,0,0.00K,08,0
26622,18,930949,2020,Flash Flood,C,7,0,0,1.00K,11,0
26623,18,930952,2020,Flood,C,89,0,0,0.00K,11,0
